In [1]:
import numpy as np
from scipy import signal
from scipy import linalg
import control
import matplotlib.pyplot as plt
import pickle

In [2]:
OSR = 256      # oversample ratio
fb = 22050     # nyquist
fs = OSR*2*fb  # sampling frequency
ts = 1/fs      # sampling period

In [3]:
file = open('filter_ss.pickle', 'rb')
[A,B,C,D] = pickle.load(file)
file.close()

## State Space Model
The shape of the state space equations determine the order of the $\Sigma\Delta$ filter. Below is the continuous time representation

$\dot{x} = A_{c}x + B_{c}u$<br>
$y = C_{c}x + D_{c}u$

The corresponding based $\delta$ model[^1].

$\delta$x = $A_{\delta}x + B_{\delta}u$<br>
$y = C_{\delta}x + D_{\delta}u$

$A_{\delta} = \dfrac{\exp(A_{c} - \textit{I})}{\Delta}$ <br>
$B_{\delta} = \dfrac{1}{\Delta}\int_{0}^{\Delta}\exp(A_{c}(t - \tau))B_{c}u(\tau)\;d\tau$ <br>
$C_{\delta} = C_{c}$ <br>
$D_{\delta} = D_{c}$ <br>

In [4]:
[A, T] = linalg.matrix_balance(A)
B = linalg.solve(T, B) 
C = C @ T

### Converting from Continuous Time to Sampled Time
c2delta - def c2delta(A,B,C,D,ts):

In [5]:
Ad = (linalg.expm(A*ts) - np.eye(A.shape[0])) / ts
Bd = np.matmul((linalg.expm(A*ts) - np.eye(A.shape[0]) ), B) / ts
Bd = np.matmul(np.linalg.inv(A), Bd)
Cd = C
Dd = D

### Structural Transformation of Filter
obsv_cst - def obsv_cst(A,B,C,D):

In [6]:
e = np.zeros((A.shape[1], 1))
e[-1] = 1
O = control.obsv(A, C)
[U, S, Vh] = linalg.svd(O)
V = Vh.T
S = np.diag(S)
S_inv = linalg.solve(S, np.eye(S.shape[0]))
T_inv = V @ S_inv @ U.conj().T
T1 = T_inv @ e;
n = A.shape[1]
q = T1.shape[1]
T0 = np.zeros((n,n))

for i in range(1, n+1):
  column = np.power(A, n-i) @ T1
  T0[:, i-1] = column[:,0]

Ad_t = linalg.solve(T0, A @ T0)
Bd_t = linalg.solve(T0, B)
Cd_t = C @ T0;
Dd_t = D;

In [7]:
[num_t, den_t] = signal.ss2tf(Ad_t,Bd_t,Cd_t,Dd_t)

### Dynamic Range Scaling of State Variable Integrators
dIIR_scaling(Ad,Bd,T0,f,ts):

In [9]:
[num_t, den_t]

[array([[ 1.00000000e-03, -1.43625903e-06,  6.46449464e+05,
          1.97524570e+07,  1.18484249e+12]]),
 array([1.00000000e+00, 1.07239893e+03, 7.16376549e+07, 3.81243671e+10,
        1.26231878e+15])]

[^1]: [$\Sigma\Delta$ Stream Computation: A New Paradigm for Low Power and High Resolution Feedback Control](https://escholarship.org/uc/item/4f46n0h6) by Poverelli, Joseph Sam <br>
[^2]: [Digital Control and Estimation: A Unified Approach](https://dl.acm.org/doi/10.5555/574885) by Richard H Middleton & 
Graham C Goodwin <br>
[^3]: [A generalized direct-form delta operator-based IIR filter with minimum noise gain and sensitivity](https://ieeexplore.ieee.org/document/933811) by Ngai Wong & Tung-Sang Ng <br>